In [1]:
import pandas as pd
import geopandas as gpd
import ipywidgets
from shapely import wkt
from IPython.display import display
from ipyleaflet import Map, GeoData, LayersControl, basemaps
from threading import Lock

def load_geodata(file_path):
    gdf = gpd.read_file(file_path)
    gdf['geometry'] = gdf['wkt'].apply(lambda x: wkt.loads(x))
    gdf.set_geometry('geometry', inplace=True)
    gdf.set_crs(epsg=4326, inplace=True)
    return gdf

def create_map(df):
    m = Map(center=(df['lat'].mean(), df['lon'].mean()), scroll_wheel_zoom=True, zoom=11, basemap=basemaps.OpenStreetMap.Mapnik)
    m.layout.height = '600px'
    return m

def handle_click(**kwargs):
    with mutex:
        properties = kwargs.get('feature').get('properties')
        grids_gdf.loc[properties['grelha_id']] = {'geometry': wkt.loads(properties['wkt'])}
        grids_gdf.set_geometry('geometry', inplace=True)
        grids_gdf.set_crs(epsg=4326, inplace=True)
        selected_grids_data.geo_dataframe = grids_gdf
        grids_gdf.to_csv('data/selected_grids.csv')

def handle_click_selected_grids(**kwargs):
    with mutex:
        id = kwargs.get('feature')['id']
        grids_gdf.drop(id, inplace=True)
        selected_grids_data.geo_dataframe = grids_gdf
        grids_gdf.to_csv('data/selected_grids.csv')

# Load your GeoDataFrame
gdf = load_geodata('data/wktComplete.csv')

# Convert the GeoDataFrame to a DataFrame
df = pd.DataFrame(gdf)

# Calculate the centroid of each polygon
df['centroid'] = gdf['geometry'].centroid

# Extract the latitude and longitude from the centroid column
df['lon'] = df['centroid'].apply(lambda point: point.x)
df['lat'] = df['centroid'].apply(lambda point: point.y)

# Create a map
m = create_map(df)

grids_gdf = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry', crs='EPSG:4326')
gdf = gdf.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

geo_data = GeoData(geo_dataframe=gdf, style={'color': 'blue', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1}, name='GeoData')
selected_grids_data = GeoData(geo_dataframe=grids_gdf, style={'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1}, name='Selected Grids')

geo_data.on_click(handle_click)
# selected_grids_data.on_click(handle_click_selected_grids)

m.add_layer(geo_data)
m.add_layer(selected_grids_data)
m.add_control(LayersControl())

m

Map(center=[38.74256107238327, -9.156505551146509], controls=(ZoomControl(options=['position', 'zoom_in_text',…